In [58]:
from env import get_db_url
import numpy as np
import pandas as pd
import os
import acquire
import prepare
import wrangle
import matplotlib.pyplot as plt
from scipy import stats
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import explore
import pydataset
from math import sqrt

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

In [85]:
zillow = wrangle.wrangle_zillow()
zillow #= zillow.drop(columns=['year_built', 'tax_amount', 'fips'])

,bedrooms,bathrooms,sqft,tax_value,year_built,tax_amount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
...,...,...,...,...,...,...,...
2152855,3.0,2.5,1809.0,405547.0,2012.0,4181.10,6059.0
2152856,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0


In [86]:
train, validate, test = prepare.split_data(zillow)
train

,bedrooms,bathrooms,sqft,tax_value,year_built,tax_amount,fips
1844941,3.0,2.0,1520.0,420429.0,1921.0,5174.91,6037.0
824995,5.0,3.0,2935.0,391054.0,1954.0,3399.33,6037.0
1214129,4.0,4.0,3394.0,1199447.0,1958.0,13254.74,6059.0
1566096,2.0,2.0,1048.0,187230.0,1925.0,3364.40,6037.0
1259406,4.0,3.0,2843.0,193625.0,2005.0,3340.72,6037.0
...,...,...,...,...,...,...,...
2036394,3.0,3.0,2162.0,230635.0,1958.0,2928.20,6037.0
551645,5.0,3.0,3054.0,324446.0,1996.0,4652.65,6037.0
1883454,4.0,2.5,2688.0,979673.0,2000.0,10920.26,6059.0
1292898,5.0,3.0,4354.0,663628.0,1997.0,7292.88,6111.0


In [87]:
train_scaled, validate_scaled, test_scaled = prepare.scale_zillow(zillow)
train_scaled

,bedrooms,bathrooms,sqft,tax_value,year_built,tax_amount,fips
0,0.12,0.062500,0.001595,0.004776,0.558140,0.004913,0.000000
1,0.20,0.093750,0.003080,0.004443,0.711628,0.003225,0.000000
2,0.16,0.125000,0.003562,0.013627,0.730233,0.012593,0.297297
3,0.08,0.062500,0.001099,0.002127,0.576744,0.003192,0.000000
4,0.16,0.093750,0.002983,0.002200,0.948837,0.003169,0.000000
...,...,...,...,...,...,...,...
1278123,0.12,0.093750,0.002269,0.002620,0.730233,0.002777,0.000000
1278124,0.20,0.093750,0.003205,0.003686,0.906977,0.004416,0.000000
1278125,0.16,0.078125,0.002821,0.011130,0.925581,0.010374,0.297297
1278126,0.20,0.093750,0.004570,0.007539,0.911628,0.006926,1.000000


In [88]:
def split_scaled_and_unscaled(df, test_size=.2, validate_size=.2, target=None, stratify_col=None):
    if stratify_col==None:
        train_validate, test = train_test_split(df, test_size = .2, random_state=823)
        train, validate = train_test_split(train_validate, test_size=validate_size / (1 - test_size), random_state=823)

        X_train = train.drop(columns=[target])
        y_train = train[target]

        X_validate = validate.drop(columns=[target])
        y_validate = validate[target]

        X_test = test.drop(columns=[target])
        y_test = test[target]

        #Set the Scaler
        scaler = sklearn.preprocessing.MinMaxScaler()
        # Note that we only call .fit with the training data,
        # but we use .transform to apply the scaling to all the data splits.
        scaler.fit(X_train)

        # Turn all the sets inot the scaled np data array
        X_train_scaled = scaler.transform(X_train)
        X_validate_scaled = scaler.transform(X_validate)
        X_test_scaled = scaler.transform(X_test)
        
        # Create a dictionary so that I can take the np arrays back to a labelled pd DataFrame
        columns = train.columns #List of Columns
        numbers = [0,1,2,3,4,5,6] #List of numbers for the scaled np array I'm converting into a dataframe
        zipped= dict(zip(numbers, columns))

        
        #turn the Train Validate and Test arrays back into labelled DataFrames
        X_train_scaled = pd.DataFrame(train_scaled).rename(columns=zipped)
        X_validate_scaled = pd.DataFrame(validate_scaled).rename(columns=zipped)
        X_test_scaled = pd.DataFrame(test_scaled).rename(columns=zipped)
        
        #Return the three scaled DataFrames
        return X_train_scaled, X_validate_scaled, X_test_scaled, X_train, y_train, X_validate, y_validate, X_test, y_test
    
    else:
        train_validate, test = train_test_split(df, test_size = .2, random_state=823, stratify=df[stratify_col])
        train, validate = train_test_split(train_validate, test_size=validate_size / (1 - test_size), random_state=823, stratify=train_validate[stratify_col])
    
        X_train = train.drop(columns=[stratify_col])
        y_train = train[stratify_col]

        X_validate = validate.drop(columns=[stratify_col])
        y_validate = validate[stratify_col]

        X_test = test.drop(columns=[stratify_col])
        y_test = test[stratify_col]

        #Set the Scaler
        scaler = sklearn.preprocessing.MinMaxScaler()
        # Note that we only call .fit with the training data,
        # but we use .transform to apply the scaling to all the data splits.
        scaler.fit(X_train)

        # Turn all the sets inot the scaled np data array
        X_train_scaled = scaler.transform(X_train)
        X_validate_scaled = scaler.transform(X_validate)
        X_test_scaled = scaler.transform(X_test)
        
        # Create a dictionary so that I can take the np arrays back to a labelled pd DataFrame
        columns = train.columns #List of Columns
        numbers = [0,1,2,3,4,5,6] #List of numbers for the scaled np array I'm converting into a dataframe
        zipped= dict(zip(numbers, columns))

        
        #turn the Train Validate and Test arrays back into labelled DataFrames
        X_train_scaled = pd.DataFrame(train_scaled).rename(columns=zipped)
        X_validate_scaled = pd.DataFrame(validate_scaled).rename(columns=zipped)
        X_test_scaled = pd.DataFrame(test_scaled).rename(columns=zipped)
        
        #Return the three scaled DataFrames
        return X_train_scaled, X_validate_scaled, X_test_scaled, X_train, y_train, X_validate, y_validate, X_test, y_test
    

In [89]:
X_train_scaled, X_validate_scaled, X_test_scaled, X_train, y_train, X_validate, y_validate, X_test, y_test = prepare.split_scaled_and_unscaled(zillow, target = 'tax_value')

In [90]:
X_train_scaled

,bedrooms,bathrooms,sqft,tax_value,year_built,tax_amount
0,0.12,0.062500,0.001595,0.558140,0.004913,0.000000
1,0.20,0.093750,0.003080,0.711628,0.003225,0.000000
2,0.16,0.125000,0.003562,0.730233,0.012593,0.297297
3,0.08,0.062500,0.001099,0.576744,0.003192,0.000000
4,0.16,0.093750,0.002983,0.948837,0.003169,0.000000
...,...,...,...,...,...,...
1278123,0.12,0.093750,0.002269,0.730233,0.002777,0.000000
1278124,0.20,0.093750,0.003205,0.906977,0.004416,0.000000
1278125,0.16,0.078125,0.002821,0.925581,0.010374,0.297297
1278126,0.20,0.093750,0.004570,0.911628,0.006926,1.000000


In [91]:
X_validate_scaled

,bedrooms,bathrooms,sqft,tax_value,year_built,tax_amount
0,0.20,0.156250,0.007634,0.869767,0.019495,0.000000
1,0.12,0.093750,0.002046,0.762791,0.005354,1.000000
2,0.12,0.062500,0.001718,0.758140,0.004639,0.000000
3,0.16,0.062500,0.002665,0.576744,0.002017,0.000000
4,0.12,0.062500,0.001361,0.744186,0.002542,0.297297
...,...,...,...,...,...,...
426038,0.16,0.093750,0.002753,0.730233,0.002546,0.000000
426039,0.08,0.031250,0.000991,0.706977,0.002542,0.000000
426040,0.12,0.062500,0.001369,0.711628,0.002668,0.000000
426041,0.16,0.093750,0.002432,0.800000,0.006536,0.297297


In [92]:
X_test_scaled 

,bedrooms,bathrooms,sqft,tax_value,year_built,tax_amount
0,0.12,0.06250,0.001645,0.860465,0.002438,0.000000
1,0.08,0.03125,0.000599,0.567442,0.001869,0.000000
2,0.12,0.06250,0.001901,0.781395,0.005711,0.297297
3,0.12,0.03125,0.000830,0.609302,0.005281,0.000000
4,0.12,0.06250,0.001309,0.846512,0.003622,0.000000
...,...,...,...,...,...,...
426038,0.16,0.09375,0.001910,0.958140,0.001520,0.000000
426039,0.12,0.06250,0.001804,0.679070,0.005331,0.000000
426040,0.16,0.06250,0.001739,0.795349,0.004267,0.297297
426041,0.08,0.06250,0.001446,0.706977,0.001829,0.000000


In [93]:
X_train

,bedrooms,bathrooms,sqft,year_built,tax_amount,fips
1844941,3.0,2.0,1520.0,1921.0,5174.91,6037.0
824995,5.0,3.0,2935.0,1954.0,3399.33,6037.0
1214129,4.0,4.0,3394.0,1958.0,13254.74,6059.0
1566096,2.0,2.0,1048.0,1925.0,3364.40,6037.0
1259406,4.0,3.0,2843.0,2005.0,3340.72,6037.0
...,...,...,...,...,...,...
2036394,3.0,3.0,2162.0,1958.0,2928.20,6037.0
551645,5.0,3.0,3054.0,1996.0,4652.65,6037.0
1883454,4.0,2.5,2688.0,2000.0,10920.26,6059.0
1292898,5.0,3.0,4354.0,1997.0,7292.88,6111.0


In [94]:
y_train

1844941     420429.0
824995      391054.0
1214129    1199447.0
1566096     187230.0
1259406     193625.0
             ...    
2036394     230635.0
551645      324446.0
1883454     979673.0
1292898     663628.0
357051      334013.0
Name: tax_value, Length: 1278128, dtype: float64

In [95]:
X_validate

,bedrooms,bathrooms,sqft,year_built,tax_amount,fips
286277,5.0,5.0,7273.0,1988.0,20516.15,6037.0
248357,3.0,3.0,1950.0,1965.0,5639.12,6111.0
691341,3.0,2.0,1638.0,1964.0,4886.81,6037.0
1995664,4.0,2.0,2540.0,1925.0,2127.97,6037.0
1559786,3.0,2.0,1297.0,1961.0,2680.64,6059.0
...,...,...,...,...,...,...
1574934,4.0,3.0,2623.0,1958.0,2684.53,6037.0
911948,2.0,1.0,945.0,1953.0,2681.04,6037.0
1696319,3.0,2.0,1305.0,1954.0,2813.74,6037.0
1441134,4.0,3.0,2318.0,1973.0,6882.54,6059.0


In [96]:
y_validate

286277     1827377.0
248357      490000.0
691341      353536.0
1995664     187716.0
1559786     219888.0
             ...    
1574934     179113.0
911948      207143.0
1696319     234710.0
1441134     592871.0
720126      535569.0
Name: tax_value, Length: 426043, dtype: float64

In [97]:
X_test

,bedrooms,bathrooms,sqft,year_built,tax_amount,fips
680956,3.0,2.0,1568.0,1986.0,2571.39,6037.0
1544220,2.0,1.0,572.0,1923.0,1972.17,6037.0
2011328,3.0,2.0,1812.0,1969.0,6014.32,6059.0
940915,3.0,1.0,792.0,1932.0,5562.09,6037.0
1199820,3.0,2.0,1248.0,1983.0,3817.24,6037.0
...,...,...,...,...,...,...
2012821,4.0,3.0,1820.0,2007.0,1605.18,6037.0
1544239,3.0,2.0,1719.0,1947.0,5614.74,6037.0
1774214,4.0,2.0,1658.0,1972.0,4495.04,6059.0
1266615,2.0,2.0,1378.0,1953.0,1930.24,6037.0


In [98]:
y_test

680956     150113.0
1544220    182323.0
2011328    548641.0
940915     466000.0
1199820    301670.0
             ...   
2012821    119830.0
1544239    475950.0
1774214    364180.0
1266615    148309.0
1303492    549341.0
Name: tax_value, Length: 426043, dtype: float64

In [99]:
# turn series into dataframes to append new columns with predicted values
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)
y_test = pd.DataFrame(y_test)

# 1. Predict based on mean
tax_value_pred_mean = y_train['tax_value'].mean()
y_train['tax_value_pred_mean'] = tax_value_pred_mean
y_validate['tax_value_pred_mean'] = tax_value_pred_mean

# 2. Do same for median
tax_value_pred_median = y_train['tax_value'].median()
y_train['tax_value_pred_median'] = tax_value_pred_median
y_validate['tax_value_pred_median'] = tax_value_pred_median

# 3.  RMSE of tax_value_pred_mean
rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_mean) ** (1/2)
rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_mean) ** (1/2)

print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2)) 

# 4.  RMSE of tax_value_pred_median
rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_median) ** (1/2)
rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_median) ** (1/2)

print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))

RMSE using Mean
Train/In-Sample:  670428.82 
Validate/Out-of-Sample:  706283.78
RMSE using Median
Train/In-Sample:  683420.19 
Validate/Out-of-Sample:  718728.08


In [100]:
#. Create the model object
lm = LinearRegression()

#. Fit to training and specify column in y_train since it is now a series
lm.fit(X_train, y_train.tax_value)

# predict
y_train['tax_value_pred_lm'] = lm.predict(X_train)

# RMSE
rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_lm) ** (1/2)

# predict validate
y_validate['tax_value_pred_lm'] = lm.predict(X_validate)

#Validate RMSE 
rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_lm) ** (1/2)

print('RMSE for OLS using LinearRegression\nTraining/In-Sample: ', rmse_train,
     '\nValidation/Out-of-Sample: ', rmse_validate)

RMSE for OLS using LinearRegression
Training/In-Sample:  88457.4824890432 
Validation/Out-of-Sample:  79404.83105952611


In [105]:
for i in range(-8,3):
    #. Create the model object
    lars = LassoLars(alpha=10**i)

    #. Fit to training and specify column in y_train since it is now a series
    lars.fit(X_train, y_train.tax_value)

    # predict
    y_train['tax_value_pred_lars'] = lars.predict(X_train)

    # RMSE
    rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_lars) ** (1/2)

    # predict validate
    y_validate['tax_value_pred_lars'] = lars.predict(X_validate)

    #Validate RMSE 
    rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_lars) ** (1/2)

    print('RMSE  using LassoLars\nTraining/In-Sample: ', rmse_train,
         '\nValidation/Out-of-Sample: ', rmse_validate,
         '\With alpha= 10^', i)

RMSE  using LassoLars
Training/In-Sample:  88457.48248904324 
Validation/Out-of-Sample:  79404.83105952619 \With alpha= 10^ -8
RMSE  using LassoLars
Training/In-Sample:  88457.48248904324 
Validation/Out-of-Sample:  79404.83105952619 \With alpha= 10^ -7
RMSE  using LassoLars
Training/In-Sample:  88457.48248904318 
Validation/Out-of-Sample:  79404.83105954203 \With alpha= 10^ -6
RMSE  using LassoLars
Training/In-Sample:  88457.4824890432 
Validation/Out-of-Sample:  79404.83105968482 \With alpha= 10^ -5
RMSE  using LassoLars
Training/In-Sample:  88457.48248904322 
Validation/Out-of-Sample:  79404.83106111268 \With alpha= 10^ -4
RMSE  using LassoLars
Training/In-Sample:  88457.48248904324 
Validation/Out-of-Sample:  79404.83107539118 \With alpha= 10^ -3
RMSE  using LassoLars
Training/In-Sample:  88457.48248904735 
Validation/Out-of-Sample:  79404.83121818057 \With alpha= 10^ -2
RMSE  using LassoLars
Training/In-Sample:  88457.48248945652 
Validation/Out-of-Sample:  79404.83264649077 \With

In [104]:
for i in range(0,4):
    # create the model object
    glm = TweedieRegressor(power=i, alpha=0)

    # fit the model to our training data. We must specify the column in y_train, 
    # since we have converted it to a dataframe from a series! 
    glm.fit(X_train, y_train.tax_value)

    # predict train
    y_train['tax_value_pred_glm'] = glm.predict(X_train)

    # evaluate: rmse
    rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_glm)**(1/2)

    # predict validate
    y_validate['tax_value_pred_glm'] = glm.predict(X_validate)

    # evaluate: rmse
    rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_glm)**(1/2)

    print("RMSE for GLM using Tweedie, power=", i, " & alpha=0\nTraining/In-Sample: ", rmse_train, 
          "\nValidation/Out-of-Sample: ", rmse_validate)

/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_glm/glm.py:284: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:289: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights


RMSE for GLM using Tweedie, power= 0  & alpha=0
Training/In-Sample:  90052.55742458206 
Validation/Out-of-Sample:  81191.10160646972
RMSE for GLM using Tweedie, power= 1  & alpha=0
Training/In-Sample:  670428.8210577338 
Validation/Out-of-Sample:  706283.7798376716


/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_glm/glm.py:284: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


RMSE for GLM using Tweedie, power= 2  & alpha=0
Training/In-Sample:  5.2641219522250995e+59 
Validation/Out-of-Sample:  2.4323041202325852e+61
RMSE for GLM using Tweedie, power= 3  & alpha=0
Training/In-Sample:  inf 
Validation/Out-of-Sample:  inf


/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/usr/local/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)


In [103]:
for i in range(0,5):
    # make the polynomial features to get a new set of features
    pf = PolynomialFeatures(degree=i)

    # fit and transform X_train_scaled
    X_train_degree2 = pf.fit_transform(X_train)

    # transform X_validate_scaled & X_test_scaled
    X_validate_degree2 = pf.transform(X_validate)
    X_test_degree2 = pf.transform(X_test)

    # create the model object
    lm2 = LinearRegression()

    # fit the model to our training data. We must specify the column in y_train, 
    # since we have converted it to a dataframe from a series! 
    lm2.fit(X_train_degree2, y_train.tax_value)

    # predict train
    y_train['tax_value_pred_poly'] = lm2.predict(X_train_degree2)

    # evaluate: rmse
    rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_poly)**(1/2)

    # predict validate
    y_validate['tax_value_pred_poly'] = lm2.predict(X_validate_degree2)

    # evaluate: rmse
    rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_poly)**(1/2)

    print("RMSE for Polynomial Model, degrees=", i, "\nTraining/In-Sample: ", rmse_train, 
          "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for Polynomial Model, degrees= 0 
Training/In-Sample:  670428.8210577338 
Validation/Out-of-Sample:  706283.7798376716
RMSE for Polynomial Model, degrees= 1 
Training/In-Sample:  88457.48248904324 
Validation/Out-of-Sample:  79404.83105952614
RMSE for Polynomial Model, degrees= 2 
Training/In-Sample:  85256.06461131039 
Validation/Out-of-Sample:  75834.97312135155
RMSE for Polynomial Model, degrees= 3 
Training/In-Sample:  83578.42769408706 
Validation/Out-of-Sample:  398264.65918852953
RMSE for Polynomial Model, degrees= 4 
Training/In-Sample:  92168.33033823494 
Validation/Out-of-Sample:  46921240.00093307
